# Explore a FiftyOne Dataset

This notebook demonstrates how to load a dataset from the FiftyOne Zoo and explore it.

We'll load the FiftyOne quickstart dataset (200 COCO images with ground-truth detections) and explore:
- Class distribution across the dataset
- Sample visualization with bounding boxes

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

# Load the quickstart dataset (200 COCO images with ground-truth detections)
dataset = foz.load_zoo_dataset("quickstart")

print(f"Dataset:      {dataset.name}")
print(f"Num samples:  {len(dataset)}")
print(f"Media type:   {dataset.media_type}")
print(f"Label fields: {dataset.get_field_schema()}")

In [ ]:
from collections import Counter

class_counts = Counter()
for sample in dataset:
    if sample.ground_truth is not None:
        for det in sample.ground_truth.detections:
            class_counts[det.label] += 1

print(f"Total detections: {sum(class_counts.values())}")
print(f"Unique classes:   {len(class_counts)}")
print("\nTop 15 classes:")
for name, count in class_counts.most_common(15):
    print(f"  {name}: {count}")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

samples = list(dataset.take(4))

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
for ax, sample in zip(axes, samples):
    img = cv2.imread(sample.filepath)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    n_dets = 0
    if sample.ground_truth is not None:
        for det in sample.ground_truth.detections:
            x, y, bw, bh = det.bounding_box
            x1, y1 = int(x * w), int(y * h)
            x2, y2 = int((x + bw) * w), int((y + bh) * h)
            cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img_rgb, det.label, (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            n_dets += 1

    ax.imshow(img_rgb)
    ax.set_title(f"{n_dets} objects")
    ax.axis("off")

plt.tight_layout()
plt.show()

## Next Steps

See `02_finetune.ipynb` to fine-tune YOLO-NAS on this dataset.